In [1]:
import pandas as pd
import numpy as np
import glob
from random import *
import os

In [6]:
class datasets(object):
    seed(10)
    def __init__(self,scale:int,strict_mode = True):
        
#         self.img_list = glob.glob(f'/mnt/wangyh/CN_patches/*/*/{scale}X/T*')
        

#         self.H_list = [i for i in self.__generator(self.img_list,label = 'H')]
#         self.L_list = [i for i in self.__generator(self.img_list,label = 'L')]
#         self.scale = scale
#         self.len = len(self.img_list)
#         self.L_len = len(self.L_list)
#         self.H_len = len(self.H_list)
        
        df = pd.read_csv('../config/patch_info.csv')
        self.H_count = df['TMB_H/L'].value_counts()['H']
        self.L_count = df['TMB_H/L'].value_counts()['L']
        
        
        self.mode = strict_mode
        
   #construct generator for dataset segmentation by label 
#     def __generator(self,img_list,label):
#         for i in img_list:
#             if i.split('/')[4] == label:
#                 yield i    
                
#     def length(self):
#         full_length = self.len
#         H_length = self.H_len
#         L_length = self.L_len
#         print(f'full length of the dataset:{full_length},including H {H_length},L {L_length}')
    
#     def __segmentation(self):
#         '''
#         train:val:train = 0.64:0.16:0.2
#         percentage of H / L should be balanced among three sets
#         '''
#         shuffle(self.H_list)
#         shuffle(self.L_list)
        
#         train_list = self.H_list[:int(self.H_len*0.64)] + self.L_list[:int(self.L_len*0.64)]
#         val_list = self.H_list[int(self.H_len*0.64):int(self.H_len*0.8)] + self.L_list[int(self.L_len*0.64):int(self.L_len*0.8)]
#         test_list = self.H_list[int(self.H_len*0.8):]+self.L_list[int(self.L_len*0.8):]
#         shuffle(train_list)
#         shuffle(val_list)
#         shuffle(test_list)
#         new_img_list = train_list+val_list+test_list
#         return train_list,val_list,test_list,new_img_list
    
#         mode = self.mode
#         if mode:
#             pass
        
#         else:
#             pass
    
#     def get_train(self):
#         return self.__segmentation()[0]
    
#     def get_val(self):
#         return self.__segmentation()[1]
    
#     def get_test(self):
#         return self.__segmentation()[2]
    
#     def get_csv(self,dir='../config/data_segmentation_csv'):
#         #notice: adjust the path for data saving
#         new_img_list = self.__segmentation()[3]
#         ground_truth = {path:path.split('/')[4] for path in new_img_list}
        
#         df = pd.DataFrame(list(ground_truth.items()))
#         #save ground truth as csv
        
#         if not os.path.exists(dir):
#             os.mkdir(dir)
#         path_csv = os.path.join(dir,f'{self.scale}X_ground_truth.csv')
# #         df.to_csv(path_csv)
        
#         #save grouping as pickle
#         grouping = {
#             'train_list':self.get_train(),
#             'val_list': self.get_val(),
#             'test_list':self.get_test()
#         }
        
#         path_grouping = os.path.join(dir,f'{self.scale}X_grouping.npy')
#         np.save(path_grouping,grouping)

    def __getattribute__(self, name: str) :
        return object.__getattribute__(self,name)

In [7]:
data_5x = datasets(5)

In [8]:
data_5x.H_count

2

In [101]:
data_5x = datasets(5)
data_10x = datasets(10)
data_20x = datasets(20)
data_40x = datasets(40)

In [102]:
data_10x.get_csv()
data_20x.get_csv()
data_40x.get_csv()

In [103]:
data_5x.get_csv()

In [13]:
data_5x.length()

full length of the dataset:28277,including H 4794,L 23483
